# 6-семинар: Scikit-learn-мен алғашқы таныстық. Қарапайым сызықтық регрессия.

**Семинар мақсаттары:**
1.  `scikit-learn` кітапханасының негізгі философиясымен танысу.
2.  `fit`, `predict` мысалында негізгі API-ді зерттеу.
3.  Деректерді оқытуға дайындауды үйрену: жүктеу, талдау, іріктемелерге бөлу (`train_test_split`).
4.  Өзіңіздің алғашқы `LinearRegression` моделіңізді оқыту.
5.  Модель сапасын метрикалар (RMSE, MAE, R²) және қалдықтарды визуалды талдау арқылы бағалау.

## 1. Scikit-Learn-ге кіріспе

**Scikit-learn** — бұл Python-дағы классикалық машиналық оқыту үшін де-факто стандарт. Оның танымалдылығы бірнеше себептерге байланысты:

*   **Бірыңғай API:** Барлық модельдер (бағалаушылар, *estimators*) ұқсас әдістерге ие: оқыту үшін `.fit()`, болжау үшін `.predict()`, бағалау үшін `.score()` және т.б. Бұл бір алгоритмді екіншісімен оңай ауыстыруға мүмкіндік береді.
*   **Кең құралдар жиынтығы:** Құрамына қажеттінің бәрі кіреді: алгоритмдер, деректерді дайындауға арналған функциялар, сапа метрикалары.
*   **Керемет құжаттама:** Көптеген мысалдармен және егжей-тегжейлі түсіндірмелермен.

### Негізгі жұмыс процесі (Estimator API)

1.  **Импорттау:** `from sklearn.module import Model`
2.  **Инстанциялау:** `model = Model(hyperparameter=value)`
3.  **Оқыту:** `model.fit(X_train, y_train)`
4.  **Болжау:** `predictions = model.predict(X_test)`
5.  **Бағалау:** `score = metric_function(y_test, predictions)`

Бүгін біз осы қадамдардың барлығын іс жүзінде өтеміз.

## 2. Міндет: Жарнама негізінде сатылымды болжау

Біз танымал `Advertising.csv` деректер жиынтығымен жұмыс істейтін боламыз, онда ТВ, радио және газеттерге жұмсалған жарнама бюджетіне байланысты өнімнің сатылымы туралы ақпарат бар.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Деректерді жүктейміз
df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/Advertising.csv')

# Алғашқы 5 жолға қараймыз
df.head()

### 2.1. Деректерді зерттеу талдауы (EDA)

Модельді оқытпас бұрын, әрқашан деректерге көзбен қарау керек. Айнымалылар арасындағы тәуелділіктерді визуалды түрде бағалау үшін pairplot құрамыз.

In [ ]:
sns.pairplot(df)
plt.show()

**pairplot-тан қорытынды:** ТВ-жарнама бюджеті (`TV`) мен сатылымдар (`sales`) арасында күшті оң сызықтық тәуелділік байқалады. `radio`-ға тәуелділік те бар, бірақ шашырауы көбірек. `newspaper` сатылымдармен ең аз байланысты болып көрінеді.

Белгілерді (X) және нысаналы айнымалыны (y) бөліп алайық.

In [ ]:
# Белгілер
X = df.drop('Sales', axis=1)

# Нысаналы айнымалы
y = df['Sales']

### 2.2. Деректерді оқыту және тест іріктемелеріне бөлу

Бұл модельді объективті бағалау үшін ең маңызды қадам. Біз оқытудан кейін моделімізді тексеру үшін деректердің бір бөлігін (тест іріктемесін) кейінге қалдырамыз. Ол үшін `train_test_split` функциясын қолданамыз.

In [ ]:
from sklearn.model_selection import train_test_split

# test_size=0.3 деректердің 30%-ы тест іріктемесіне кететінін білдіреді
# random_state=42 нәтиженің қайталанбалылығын қамтамасыз етеді (бірдей "кездейсоқ" бөлу)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 3. Сызықтық регрессия моделін оқыту

In [ ]:
from sklearn.linear_model import LinearRegression

# 1-қадам: Модельді инстанциялау
model = LinearRegression()

# 2-қадам: Модельді оқыту деректерінде оқыту
model.fit(X_train, y_train)

### 3.1. Коэффициенттерді интерпретациялау

Модель оқытылды! Ол біздің регрессия теңдеуіміз үшін оңтайлы коэффициенттерді тапты. Оларға қарап көрейік.

In [ ]:
print("Бос мүше (intercept, w0):", model.intercept_)

# Салмақтарды ыңғайлы қарау үшін DataFrame құрамыз
coeffs = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nӘр белгі үшін салмақтар (w1, w2, ...):")
print(coeffs)

**Мұны қалай түсіндіруге болады?**
*   **Intercept (2.85):** Егер біз жарнамаға бірде-бір теңге жұмсамасақ, біздің модель сатылымды 2.69 бірлік деңгейінде болжайды.
*   **TV (0.04):** ТВ-жарнамаға салынған әрбір қосымша теңге, басқалары тең болған жағдайда, сатылымды 0.04 бірлікке арттырады.
*   **radio (0.199):** Радио-жарнамаға жұмсалған әрбір қосымша теңге 0.199 бірлік өсім береді.
*   **newspaper (0.0001):** Газеттердің әсері, модельдің пікірінше, минималды.

## 4. Модель сапасын бағалау

Енді модель оқытылғаннан кейін, оның сапасын модель әлі көрмеген **тест іріктемесінде** бағалайық.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 3-қадам: Тест деректерінде болжамдар жасаймыз
y_pred = model.predict(X_test)

# 4-қадам: Сапаны метрикалар көмегімен бағалаймыз
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE (Mean Absolute Error): {mae:.2f}")
print(f"MSE (Mean Squared Error): {mse:.2f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R^2 (Детерминация коэффициенті): {r2:.2f}")

**Метрикаларды интерпретациялау:**
*   **RMSE (1.93):** Орташа есеппен, біздің модель сатылымды болжауда 1.95 бірлікке қателеседі. Орташа сатылым шамамен 14 екенін ескерсек, бұл жаман нәтиже емес.
*   **R² (0.86):** Біздің модель сатылым деректеріндегі өзгергіштіктің (дисперсияның) 86%-ын түсіндіреді. Бұл өте жақсы көрсеткіш.

### 4.1. Қалдықтарды талдау

Модель қателерінде қандай да бір жасырын заңдылықтардың жоқтығын тексерейік. Идеалды жағдайда қалдықтар нөл айналасында кездейсоқ таралуы керек.

In [ ]:
residuals = y_test - y_pred

sns.scatterplot(x=y_test, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Қалдықтар графигі')
plt.xlabel('y_test нақты мәндері')
plt.ylabel('Қалдықтар')
plt.show()

**Қорытынды:** График жақсы көрінеді. Нүктелер айқын үлгілерсіз (мысалы, парабола) ретсіз таратылған, бұл сызықтық модельдің осы жағдайда адекватты екенін көрсетеді.

### 4.2. Қалдықтарды тереңірек талдау: тәуелділіктерді іздеу

Тағы бір пайдалы диагностикалық құрал — әрбір бастапқы белгіге қатысты қалдықтар графиктерін құру. Егер модель белгіден барлық ақпаратты толығымен "сіңірген" болса, онда қателерде осы белгімен байланысты ешқандай құрылым қалмауы керек.

In [ ]:
# Үш графигі бар фигура құрамыз
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Әр белгі бойынша қалдықтар графиктері', fontsize=16)

# Қалдықтар vs TV
sns.scatterplot(ax=axes[0], x=X_test['TV'], y=residuals)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('TV бюджеті')
axes[0].set_ylabel('Қалдықтар')

# Қалдықтар vs Radio
sns.scatterplot(ax=axes[1], x=X_test['Radio'], y=residuals)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Radio бюджеті')
axes[1].set_ylabel('')

# Қалдықтар vs Newspaper
sns.scatterplot(ax=axes[2], x=X_test['Newspaper'], y=residuals)
axes[2].axhline(y=0, color='r', linestyle='--')
axes[2].set_xlabel('Newspaper бюджеті')
axes[2].set_ylabel('')

plt.show()

**Қорытынды:** Барлық үш жағдайда да қалдықтар нөл айналасында біркелкі таратылған кездейсоқ "шу" сияқты көрінеді. Бұл қарапайым сызықтық модельдің осы деректерге жақсы сәйкес келетінін және қандай да бір айқын сызықтық емес тәуелділіктерді жіберіп алмайтынын тағы да растайды. Егер, мысалы, "Қалдықтар vs TV" графигінде біз параболаны көрген болсақ, бұл модельге `TV^2` белгісін қосу керектігінің күшті сигналы болар еді.

## Қорытынды

Бұл семинарда біз `scikit-learn` көмегімен қарапайым машиналық оқыту моделін құрудың толық циклінен өттік: деректерді жүктеуден бастап, оқыту, бағалау және диагностикалауға дейін. Біз сызықтық регрессияның осы міндетті жақсы орындайтынына көз жеткіздік.